In [14]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import nltk
import string
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

# Initialize and load dataset

We split the dataset into training, validation and testing datasets. We use the training data to train and fine-tune our model and the testing set to finally evaulate our model.

The entire dataset is split into the training set (70%), validation set (15%), and test set (15%).
Also, when evaluating text classifiers on the 20 Newsgroups data, we strip newsgroup-related metadata.The F-score will be lower because it is more realistic.

In [2]:
#Remove metadata 
remove = ('headers', 'footers', 'quotes') 
RANDOM_STATE = 10

# Load dataset
print("Loading 20 newsgroups dataset")
newsdata = fetch_20newsgroups(subset='all')
len(newsdata.data)

Loading 20 newsgroups dataset


18846

We split the data and sample the unlabelled data in a stratified manner

In [3]:
X_remaining, X_test, Y_remaining, Y_test = train_test_split(newsdata.data, newsdata.target, test_size=0.20, random_state=RANDOM_STATE)
sss = StratifiedShuffleSplit(n_splits=1, test_size= 0.625,random_state=RANDOM_STATE)
X_remaining=np.array(X_remaining)
Y_remaining=np.array(Y_remaining)
for train_index, test_index in sss.split(X_remaining,Y_remaining):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_unlabel = X_remaining[train_index], X_remaining[test_index]
    y_train, y_unlabel = Y_remaining[train_index], Y_remaining[test_index]
    
print('Data loaded')
print()
print('Training data documents:', len(X_train))
print('Development data documents:', len(X_unlabel))
print('Test data documents:', len(X_test))
print()
print('Total Newsgroups :', newsdata.target_names)

TRAIN: [ 4830  5487  1131 ...  6543 10054  9924] TEST: [ 2647  7876  4857 ... 13387  2034  8962]
Data loaded

Training data documents: 5653
Development data documents: 9423
Test data documents: 3770

Total Newsgroups : ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


We save the data

In [6]:
print ("Saving data")
np.save('data/train_data.npy', X_train)
np.save('data/dev_data.npy', X_unlabel)
np.save('data/test_data.npy', X_test)

np.save('data/train_label.npy', y_train)
np.save('data/dev_label.npy', y_unlabel)
np.save('data/test_label.npy', Y_test)

print ("Data saved!")

Saving data
Data saved!


## Count vectorizer

We establish a baseline model by using a simple CountVectorizer to extract features without any preprocessing. We perform classification using Multinomial Naive-Bayes.

In [7]:
NB1 = CountVectorizer(analyzer= 'word')
data_train = NB1.fit_transform(X_train)

data_dev = NB1.transform(X_unlabel)
classifier_NB = MultinomialNB(alpha=0.01)
classifier_NB.fit(data_train, y_train)
pred = classifier_NB.predict(data_dev)
print("NB classifier accuracy: {}".format(round(metrics.accuracy_score(y_unlabel, pred),4)))

NB classifier accuracy: 0.845


In [13]:
with open('model/NB_init.pkl', 'wb') as fid:
    pickle.dump(classifier_NB, fid) 

We check if the unlabelled data has been sampled in a stratified manner. This can be seen by looking at the number of datapoints belonginig to each category.

In [18]:
unique_elements, counts_elements = np.unique(y_unlabel, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19]
 [409 488 481 486 488 483 491 505 505 493 504 495 480 489 478 508 456 484
  384 316]]
